<a href="https://colab.research.google.com/github/Shubh10501/Social-Distancing-Detector/blob/main/Social_distance_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The Sparks Foundation** 
## Name - Shubhankar Kalekar
## *Task 3 - Social Distancing Detector*


> To find the violation of social distancing rules in the given video


In [ ]:
#!nvidia-smi

## **Variable Values**

In [ ]:
# initialize minimum probability to filter weak detections along with
# the threshold when applying non-maxima suppression
MIN_CONF = 0.3
NMS_THRESH = 0.3

# Minimun safe distance according to rules between people (in pixels)
MIN_DISTANCE = 50

## **Detection Function**

In [ ]:
#Packages and modules
import numpy as np
import cv2

def detect_people(frame, net, ln, personIdx=0):
	# grab the dimensions of the frame and  initialize the list of
	# results
	(H, W) = frame.shape[:2]
	results = []

	# construct a blob from the input frame and then perform a forward
	# pass of the YOLO object detector, giving us our bounding boxes
	# and associated probabilities
	blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
		swapRB=True, crop=False)
	net.setInput(blob)
	layerOutputs = net.forward(ln)

	# initialize our lists of detected bounding boxes, centroids, and
	# confidences, respectively
	boxes = []
	centroids = []
	confidences = []

	# loop over each of the layer outputs
	for output in layerOutputs:

		# loop over each of the detections
		for detection in output:

			# Confidence level of current object in the frame
			scores = detection[5:]
			classID = np.argmax(scores)
			confidence = scores[classID]

			# Criterias for detection -
      #(1) ensuring that the object detected was a person 
      #(2) that the minimum confidence is met

			if classID == personIdx and confidence > MIN_CONF:

				# scale the bounding box coordinates back relative to
				# the size of the image, keeping in mind that YOLO
				# actually returns the center (x, y)-coordinates of
				# the bounding box followed by the boxes' width and
				# height
				box = detection[0:4] * np.array([W, H, W, H])
				(centerX, centerY, width, height) = box.astype("int")

        # Derivation of top and left corner using the x and y coordinates
				x = int(centerX - (width / 2))
				y = int(centerY - (height / 2))

				# update our list of bounding box coordinates,
				# centroids, and confidences
				boxes.append([x, y, int(width), int(height)])
				centroids.append((centerX, centerY))
				confidences.append(float(confidence))

  # Supression of weak and overlapping boxes using non-maxima suppression
	idxs = cv2.dnn.NMSBoxes(boxes, confidences, MIN_CONF, NMS_THRESH)



	# Atleast one detection exists
	if len(idxs) > 0:
		# Loop over the indexes we are keeping
		for i in idxs.flatten():
			# Extraction of bounding box coordinates
			(x, y) = (boxes[i][0], boxes[i][1])
			(w, h) = (boxes[i][2], boxes[i][3])

			# update our results list to consist of the person
			# prediction probability, bounding box coordinates,
			# and the centroid
			r = (confidences[i], (x, y, x + w, y + h), centroids[i])
			results.append(r)

	return results

## **Finding the violations in given video frame-by-frame**

In [ ]:
# USAGE
# python social_distance_detector.py --input pedestrians.mp4
# python social_distance_detector.py --input pedestrians.mp4 --output output.avi

# Modules and packages
from google.colab.patches import cv2_imshow
from scipy.spatial import distance as dist
import numpy as np
import argparse
import imutils
import cv2
import os

# construct the argument parse and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--input", type=str, default="",
	help="path to (optional) input video file")
ap.add_argument("-o", "--output", type=str, default="",
	help="path to (optional) output video file")
ap.add_argument("-d", "--display", type=int, default=1,
	help="whether or not output frame should be displayed")
args = vars(ap.parse_args(["--input","/content/drive/MyDrive/Social Distancing /pedestrians.mp4","--output","my_output.avi","--display","1"]))

# Load the COCO class labels our YOLO model was trained on
labelsPath = os.path.sep.join(["/content/drive/MyDrive/Social Distancing /yolo-coco/coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")

# Derive the paths to the YOLO weights and model configuration
weightsPath = os.path.sep.join(["/content/drive/MyDrive/Social Distancing /yolo-coco/yolov3.weights"])
configPath = os.path.sep.join(["/content/drive/MyDrive/Social Distancing /yolo-coco/yolov3.cfg"])

# load our YOLO object detector trained on COCO dataset (80 classes)
print("Loading YOLO ...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

# determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# initialize the video stream and pointer to output video file
print("Accessing the video stream...")
vs = cv2.VideoCapture(args["input"] if args["input"] else 0)
writer = None

# Loop over the frames from the video stream
while True:
	# Read the next frame from the file
	(grabbed, frame) = vs.read()

	# Breaking the loop when the frames end
	if not grabbed:
		break

	# Resize the frame and then detect people (and only people) in it
	frame = imutils.resize(frame, width=700)
	results = detect_people(frame, net, ln,
		personIdx=LABELS.index("person"))

	# Initialize the set of indexes that violate the minimum social
	# distance
	violate = set()

	# Ensuring that there are atleast 2 people in the frame
	if len(results) >= 2:
		# Extract all centroids from the results and compute the
		# Euclidean distances between all pairs of the centroids
		centroids = np.array([r[2] for r in results])
		D = dist.cdist(centroids, centroids, metric="euclidean")

		# Loop over the upper triangular of the distance matrix
		for i in range(0, D.shape[0]):
			for j in range(i + 1, D.shape[1]):
				# Check if the distance between two people is less than
        # specified
				if D[i, j] < MIN_DISTANCE:
					# Update our violation set with the indexes of
					# the centroid pairs
					violate.add(i)
					violate.add(j)

	# Loop over the results
	for (i, (prob, bbox, centroid)) in enumerate(results):
		# Extract the bounding box and centroid coordinates, then
		# initialize the color of the annotation
		(startX, startY, endX, endY) = bbox
		(cX, cY) = centroid
		color = (0, 255, 0)

		# If the index pair exists within the violation set, then
		# update the color
		if i in violate:
			color = (0, 0, 255)

		# Draw -
    #(1) a bounding box around the person 
    #(2) the centroid coordinates of the person,
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
		cv2.circle(frame, (cX, cY), 5, color, 1)

	# Total number of violations per frame
	text = "Violations: {}".format(len(violate))
	cv2.putText(frame, text, (10, frame.shape[0] - 25),
		cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 3)

	# Check if the output frame is displayed on our
	# screen
	if args["display"] > 0:
		
		cv2_imshow(frame)
		key = cv2.waitKey(1) & 0xFF

		# Exiting the loop 
		if key == ord("q"):
			break

	# If an output video file path has been supplied and the video
	# writer has not been initialized, do so now
	if args["output"] != "" and writer is None:
		# Initialize our video writer
		fourcc = cv2.VideoWriter_fourcc(*"MJPG")
		writer = cv2.VideoWriter(args["output"], fourcc, 25,
			(frame.shape[1], frame.shape[0]), True)

	# if the video writer is not None, write the frame to the output
	# video file
	if writer is not None:
		writer.write(frame)